In [1]:
import pandas as pd
from Bio import SeqIO
from collections import defaultdict
import scipy.io as sio


input:dataname and gbk address

In [2]:

csv_file = 'example.csv'

gbk_df = pd.read_csv(csv_file)
unique_aSDomain = set() 
gbk_data = [] 

In [3]:
for idx, row in gbk_df.iterrows():
    gbk_file = row['gbk_address']  
    id_value = row['dataname'] 
    with open(gbk_file, 'r') as file:
        for record in SeqIO.parse(file, 'genbank'):
           
            cds_features = []
            for feature in record.features:
                if feature.type == 'CDS' and 'translation' in feature.qualifiers:
                    cds_features.append(feature)
            
           
            for feature in record.features:
                if feature.type == 'PFAM_domain':
                    if 'aSDomain' in feature.qualifiers and 'translation' in feature.qualifiers:
                        aSDomain = feature.qualifiers['aSDomain'][0]
                        domain_translation = feature.qualifiers['translation'][0]
                        
                        
                        parent_cds = None
                        for cds_feature in cds_features:
                            
                            if (feature.location.start >= cds_feature.location.start and 
                                feature.location.end <= cds_feature.location.end):
                                parent_cds = cds_feature
                                break
                        
                        
                        if parent_cds:
                            gene_translation = parent_cds.qualifiers['translation'][0]
                            
                            
                            gene_name = parent_cds.qualifiers.get('gene', ['Unknown'])[0]
                            locus_tag = parent_cds.qualifiers.get('locus_tag', ['Unknown'])[0]
                            
                         
                            gbk_data.append({
                                'id': id_value, 
                                'aSDomain': aSDomain, 
                                'domain_translation': domain_translation,
                                'gene_translation': gene_translation,
                                'gene_name': gene_name,
                                'locus_tag': locus_tag
                            })
                        else:
                         
                            gbk_data.append({
                                'id': id_value, 
                                'aSDomain': aSDomain, 
                                'domain_translation': domain_translation,
                                'gene_translation': None,
                                'gene_name': 'Not found',
                                'locus_tag': 'Not found'
                            })
                        
                      
                        unique_aSDomain.add(aSDomain)



with open('unique_aSDomain.txt', 'w') as f:

    for domain in unique_aSDomain:
        f.write(f"{domain}\n")


gbk_data_df = pd.DataFrame(gbk_data)
gbk_data_df.to_csv('siderophore_BGC_all_domain_info.csv', index=False)


In [3]:
###get domain occurance
for idx, row in gbk_df.iterrows():
    gbk_file = row['gbk_address'] 
    id_value = row['dataname']
    with open(gbk_file, 'r') as file:
        for record in SeqIO.parse(file, 'genbank'):
           
            cds_features = []
            for feature in record.features:
                if feature.type == 'CDS' and 'translation' in feature.qualifiers:
                    cds_features.append(feature)
            
            
            for feature in record.features:
                if feature.type == 'PFAM_domain':
                    if 'aSDomain' in feature.qualifiers and 'translation' in feature.qualifiers:
                        aSDomain = feature.qualifiers['aSDomain'][0]
                        domain_translation = feature.qualifiers['translation'][0]
                        
                       
                        parent_cds = None
                        for cds_feature in cds_features:
                            
                            if (feature.location.start >= cds_feature.location.start and 
                                feature.location.end <= cds_feature.location.end):
                                parent_cds = cds_feature
                                break
                        
                        
                        if parent_cds:
                            gene_translation = parent_cds.qualifiers['translation'][0]
                            
                            
                            gene_name = parent_cds.qualifiers.get('gene', ['Unknown'])[0]
                            locus_tag = parent_cds.qualifiers.get('locus_tag', ['Unknown'])[0]
                            
                            
                            gbk_data.append({
                                'id': id_value, 
                                'aSDomain': aSDomain, 
                                'domain_translation': domain_translation,
                                'gene_translation': gene_translation,
                                'gene_name': gene_name,
                                'locus_tag': locus_tag
                            })
                        else:
                            
                            gbk_data.append({
                                'id': id_value, 
                                'aSDomain': aSDomain, 
                                'domain_translation': domain_translation,
                                'gene_translation': None,
                                'gene_name': 'Not found',
                                'locus_tag': 'Not found'
                            })
                        
                       
                        unique_aSDomain.add(aSDomain)


with open('all_siderophoreBGC_unique_aSDomain.txt', 'w') as f:

    for domain in unique_aSDomain:
        f.write(f"{domain}\n")


gbk_data_df = pd.DataFrame(gbk_data)
gbk_data_df.to_csv('all_siderophore_BGC_all_domain_info.csv', index=False)